---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [33]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

import re
from datetime import datetime as dtt

def extractDates(dt):
    
    
    # To extract "04/20/2009; 04/20/09; 4/20/09; 4/3/09" formats
    
    retVal = re.findall(r'(\d?\d[/-][\d?\d][/-]\d\{4})',dt)
    
    if(len(retVal) != 0):
        #print("1st format",retVal)
        return dtt.strftime(dt.strptime(retVal[0],'%m/%d/%Y'),'%m/%d/%y')
    
    retVal = re.findall(r'(\d?\d[/-]\d?\d[/-]\d{2})',dt)
    
    if(len(retVal) != 0):
        for i in retVal:
            j = i.split('/')
            if (len(j) == 3):
                if not int(i.split('/')[1]) > 31:
                    #print("2nd format with /",i)
                    return i
            else:
                #print("2nd format with -",i)
                #return dt.strftime(dt.strptime(i,'%m-%d-%Y'),'%m/%d/%y')
                return dtt.strftime(dtt.strptime(i,'%m-%d-%y'),'%m/%d/%y')
    
    # To extract "Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;" formats
    
    retVal = re.findall(r'([Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|May|June|July|August|September|October|November|December]+[\s|-|.|,]+\d?\d[\s|-|.|,]+\d{4})',dt)
    if(len(retVal) != 0):
        #print("2nd format",retVal)
        if '-' in retVal[0]:
            return dtt.strftime(dtt.strptime(retVal[0],'%b-%d-%Y'),'%m/%d/%y')
        elif '.' in retVal[0]:
            if len(retVal[0].split('.')[0]) == 3:
                return dtt.strftime(dtt.strptime(retVal[0],'%b. %d, %Y'),'%m/%d/%y')
            else:
                return dtt.strftime(dtt.strptime(retVal[0],'%B. %d, %Y'),'%m/%d/%y')
        elif ',' in retVal[0]:
            if len(retVal[0].split(' ')[0]) == 3:
                return dtt.strftime(dtt.strptime(retVal[0],'%b %d, %Y'),'%m/%d/%y')
            else:
                return dtt.strftime(dtt.strptime(retVal[0],'%B %d, %Y'),'%m/%d/%y')
        else:
            if len(retVal[0].split(' ')[0]) == 3:
                return dtt.strftime(dtt.strptime(retVal[0],'%b %d %Y'),'%m/%d/%y')
            else:
                return dtt.strftime(dtt.strptime(retVal[0],'%B %d %Y'),'%m/%d/%y')
    # To extract "20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009" formats
    
    retVal = re.findall(r'(\d?\d\s[Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|May|June|July|August|September|October|November|December]+[\s|-|.|,]+\d{4})',dt)
    if(len(retVal) != 0):
        #print("3rd format",retVal)
        return retVal[0]
        
    # To extract "Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009" formats
    
    retVal = re.findall(r'([Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec]+\s\d?\d[th|st|nd|rd]{2},\s\d{4})',dt)
    if(len(retVal) != 0):
        #print("4th format",retVal)
        return retVal[0]
    
    # To extract "Feb 2009; Sep 2009; Oct 2010" formats
    
    retVal = re.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)+\s+\d{4}',dt)
    if(len(retVal) != 0):
        #print("5th format",retVal)
        return retVal[0]
    
    # To extract "6/2008; 12/2009" formats
    
    retVal = re.findall(r'\d?\d/\d{4}',dt)
    if(len(retVal) != 0):
        #print("6th format",retVal)
        return '01/'+retVal[0]
    
    # To extract "2009; 2010" formats
    
    retVal = re.findall(r'\d{4}',dt)
    if(len(retVal) != 0):
        #print("7th format",retVal)
        return '01/01/'+retVal[0]
    
    return "No date extracted"

#df2 = df.str.findall(r'(\d?\d[/-]\d?\d[/-]\d\d\d\d)|(\d?\d[/-]\d?\d[/-]\d\d)|(\d?\d[/-]\d\d\d\d)|([Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|May|June|July|August|September|October|November|December]+[\s|-|.|,]+\d?\d[\s|-|.|,]+\d\d\d\d)|(\d?\d\s[Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|May|June|July|August|September|October|November|December]+[\s|-|.|,]+\d\d\d\d)|([Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec]+\s\d?\d[th|st|nd|rd]{2},\s\d\d\d\d)')
#list(df2)

df2 = df.apply(extractDates)
#dict1 = {'Original Text':df,'ExtractedDate':df2}
#df3 = pd.DataFrame(dict1)

#import datetime as dt

#[dt.datetime(x) for x in df2]

ValueError: time data 'Age 16, 1991' does not match format '%b %d, %Y'

In [27]:
import re
dt = "Jln 16, 1991 Age 19,1990 February 18, 1992"
retVal = re.findall(r'\b?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|May|June|July|August|September|October|November|December\b[\s|-|.|,]+\d?\d[\s|-|.|,]+\d{4})',dt)
retVal

['Jan', 'Feb']

In [3]:
def date_sorter():
    
    # Your code here
    
    return # Your answer here